In [ ]:
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
plt.ioff()

In [ ]:
def preprocess(fname):
    contents = open(fname, "r").read()
    mapstr, movestr = contents.split("\n\n")
    robomap = []
    moves = []
    for line in mapstr.split("\n"):
        robomap.append(line)

    moves = "".join([line for line in movestr.split("\n")])
    robomap = np.genfromtxt(fname=StringIO(mapstr), delimiter=1, dtype=str, comments=None)
    return robomap, moves

def check_ahead(robomap, coords, move):
    check_coords = list(coords)
    if move == ">":
        check_coords[1] += 1
    elif move == "<":
        check_coords[1] -= 1
    elif move == "^":
        check_coords[0] -= 1
    elif move == "v":
        check_coords[0] += 1
    
    return robomap[check_coords[0]][check_coords[1]], check_coords

def make_move(robomap, coords, move):
    move_what = robomap[coords[0]][coords[1]]
    intheway, newcoords = check_ahead(robomap, coords, move)
    assert intheway == "."
    robomap[coords[0]][coords[1]] = "." # free up current position
    robomap[newcoords[0]][newcoords[1]] = move_what # move robot or crate to new position
    return robomap

def part1(robomap, moves):
    for move in moves:
        # print(move)
        robot_coords = list(zip(*(robomap=="@").nonzero()))[0]
        needs_to_move = [robot_coords]

        while needs_to_move:
            check, check_coords = check_ahead(robomap, needs_to_move[-1], move)
            if check == ".":
                make_move(robomap, needs_to_move[-1], move)
                del needs_to_move[-1]
                # print(robomap)
            elif check == "#":
                needs_to_move = []  # nothing moves
            elif check == "O":
                needs_to_move.append(check_coords)
    
    row, col = (robomap=="O").nonzero()
    sol = 100 * sum(row) + sum(col)
    return sol

robomap, moves = preprocess("day15_example.txt")

part1_example_sol = part1(robomap, moves)

print(f"Part 2 solution for example data: {(part1_example_sol)}")
assert (part1_example_sol) == 10092

In [ ]:
robomap, moves = preprocess("day15_input.txt")

part1_sol = part1(robomap, moves)

print(f"Part 1 solution: {(part1_sol)}")

In [ ]:
def preprocess_part2(fname):
    contents = open(fname, "r").read()
    mapstr, movestr = contents.split("\n\n")
    robomap = []
    moves = []
    for line in mapstr.split("\n"):
        newline = []
        for s in line:
            if s == "O":
                newline.extend(["[", "]"])
            elif s == "#":
                newline.extend(["#", "#"])
            else:
                newline.extend([s, "."])
        newline = "".join(newline)
        robomap.append(newline)
    moves = "".join([line for line in movestr.split("\n")])
    robomap = np.genfromtxt(fname=StringIO("\n".join(robomap)), delimiter=1, dtype=str, comments=None)
    return robomap, moves

def printmap(robomap):
    for l in robomap:
        print("".join(l))

def can_move(current_obj, robomap, coords, move):
    if current_obj == "@":
        check, check_coords =  check_ahead(robomap, coords, move)
        if check == ".":
            return True
        elif check == "#":
            return False
        elif check == "]":
            return can_move("crate", robomap, (check_coords[0], check_coords[1] - 1), move)
        elif check == "[":
            return can_move("crate", robomap, check_coords, move) 
            
    elif current_obj == "crate":
        check, check_coords = check_ahead(robomap, coords, move)
        check2, check_coords2 = check_ahead(robomap, (coords[0], coords[1] + 1), move)
        if move == "^" or move == "v":
            if "#" in (check, check2):
                return False # wall above/below
            elif check == check2 == ".":
                return True # free to move
            elif check == "]" and check2 == "[":
                return can_move("crate", robomap, (check_coords[0], check_coords[1] - 1), move) and can_move("crate", robomap, check_coords2, move)
            elif check == "[":
                return can_move("crate", robomap, check_coords, move)
            elif check == "]":
                return can_move("crate", robomap, (check_coords[0], check_coords[1] - 1), move)
            elif check2 == "[":
                return can_move("crate", robomap, check_coords2, move)
        if move == "<":
            if check == "#":
                return False # wall to the left
            elif check == "]":
                return can_move("crate", robomap, (check_coords[0], check_coords[1] - 1), move)  # crate to the left
            elif check == ".":
                return True
        if move == ">":
            if check2 == "#":
                return False # wall to the right
            elif check2 == "[":
                return can_move("crate", robomap, check_coords2, move)   # crate to the right
            elif check2 == ".":
                return True # free to the right
                                       
def check_ahead(robomap, coords, move):
    check_coords = list(coords)
    if move == ">":
        check_coords[1] += 1
    elif move == "<":
        check_coords[1] -= 1
    elif move == "^":
        check_coords[0] -= 1
    elif move == "v":
        check_coords[0] += 1
    return robomap[check_coords[0]][check_coords[1]], tuple(check_coords)

def make_move(move_what, robomap, coords, move):
    if move_what == "@":
        check, newcoords = check_ahead(robomap, coords, move)
        if move == "^" or move == "v":
            if check == "[":
                robomap = make_move("crate", robomap, newcoords, move)
            elif check == "]":
                robomap = make_move("crate", robomap, (newcoords[0], newcoords[1] - 1), move)
        elif move == "<":
            if check == "]":
                robomap = make_move("crate", robomap, (newcoords[0], newcoords[1] - 1), move)
        elif move == ">":
            if check == "[":
                robomap = make_move("crate", robomap, newcoords, move)       

        robomap[coords[0]][coords[1]] = "." # free up current position
        robomap[newcoords[0]][newcoords[1]] = "@" # move robot to new position
        return robomap

    elif move_what == "crate":
        check, newcoords = check_ahead(robomap, coords, move)
        check2, newcoords2 = check_ahead(robomap, (coords[0], coords[1] + 1), move)
        if move == "^" or move == "v":
            if check == check2 == ".":
                pass
            elif check == "]" and check2 == "[":
                robomap = make_move("crate", robomap, (newcoords[0], newcoords[1] - 1), move)
                robomap = make_move("crate", robomap, newcoords2, move)
            elif check == "[":
                robomap = make_move("crate", robomap, newcoords, move)
            elif check == "]":
                robomap = make_move("crate", robomap, (newcoords[0], newcoords[1] - 1), move)
            elif check2 == "[":
                robomap = make_move("crate", robomap, newcoords2, move)
        if move == "<":
            if check == "]":
                robomap = make_move("crate", robomap, (newcoords[0], newcoords[1] - 1), move)  # crate to the left
        if move == ">":
            if check2 == "[":
                robomap = make_move("crate", robomap, newcoords2, move)   # crate to the right
     
        robomap[coords[0]][coords[1]] = "." # free up current position
        robomap[coords[0]][coords[1] + 1] = "." # free up current position
        robomap[newcoords[0]][newcoords[1]] = "[" # move crate to new position
        robomap[newcoords2[0]][newcoords2[1]] = "]" # move crate to new position   
        return robomap

def f(x):
    return ord(x)

def savemap(robomap, midx, move):
    imh.set_data(np.vectorize(f)(robomap))
    ax.set_title(move)
    fig.savefig(f"{midx:04d}.png", dpi=200)
    # plt.close(fig)

def part2(robomap, moves):
    for midx, move in enumerate(moves):
        # savemap(robomap, midx, move)
        robot_coords = list(zip(*(robomap=="@").nonzero()))[0]

        if can_move("@", robomap, robot_coords, move):
            robomap = make_move("@", robomap, robot_coords, move)

    row, col = (robomap=="[").nonzero()
    sol = 100 * sum(row) + sum(col)
    printmap(robomap)
    return sol


robomap, moves = preprocess_part2("day15_example.txt")
# printmap(robomap)
# print(can_move("crate", robomap, (1,6), "^"))
# print(can_move("crate", robomap, (1,6), "v"))
# print(can_move("crate", robomap, (1,16), ">"))
# print(can_move("crate", robomap, (7,14), "^"))
# print(can_move("crate", robomap, (7,14), ">"))

# plotting
r, c = robomap.shape
xticks = np.arange(0,c)
yticks = np.arange(0,r)

fig, ax = plt.subplots(figsize=(12,5));
imh = ax.imshow(np.vectorize(f)(robomap));
ax.set_aspect('equal', 'box')
ax.set_xticks([])
ax.set_yticks([])

part2_example_sol = part2(robomap, moves)

print(f"Part 2 solution for example data: {(part2_example_sol)}")
assert (part2_example_sol) == 9021


In [ ]:
robomap, moves = preprocess_part2("day15_input.txt")

part2_sol = part2(robomap, moves)

print(f"Part 2 solution : {(part2_sol)}")